# DSCI 6001 5.4 Lab QR Factorization III

### Householder Factorization


## Rotation Strategies for QR Factorization


The Gram-Schmidt algorithm is not terribly efficient for heavy repeated applications , however, and so **rotation strategies** are what are typically applied. These are either **Householder reflections** or **Givens rotations**. These strategies are typically employed in the "Francis" algorithm which is the current state of the art for square matrix decomposition.  


A simple tweak is the usual way that the QR decomposition is presented in practice:

Consider:

$A = QR$

now reverse the order of multiplication:

$RQ = Q^{-1}AQ$

It so happens, due to reasons not covered in this course, that multiple applications of the similarity transformation result in a final product $RQ$ that becomes upper triangular such that the eigenvalues can be read from the diagonal:

$A = Q_{k}Q_{k-1}Q_{k-2}...Q_{1}R$  - k is the number of columns for original matrix

The typical way this above formulation is done is by using **rotations**; either **Householder** or **Givens rotations.** We will only cover Householder rotations today due to time constraints required in developing an intuition for Givens rotations. You are encouraged to explore this on your own.

## Householder reflections

Householder reflections are a way of obtaining $Q$ implicitly without ever calculating the GS basis directly. They are efficient to calculate using common matrix operations and can be improved using modern techniques of broadcasting. They do not require explicit calculations of matrix products and require less in the way of storage and operations than any of the GS algorithms.

### Householder reflections: Concepts

Householder transformations are generalizations of reflections across the plane. These are matrices of the form:

$$ H_{u} = I - 2uu^{T}$$

Where $I$ is the identity matrix, and $u$ is an $N$ dimensional unit vector.

H is symmetric: $(H_{u})^{T} = (I - 2uu^{T})^{T} = I^{T}-2(uu^{T})^{T} = I - 2uu^{T} = H_{u}$

H is also orthogonal: $H_{u}^{T}H_{u} = (I - 2uu^{T})^{T}(I - 2uu^{T}) = I - 4uu^{T} + 4uu^{T}uu^{T} = I$

When you apply $H_{u}$ to a target vector $\bf{y}$: 

$H_{u}{\bf{y}} = {\bf{y}}-2uu^{T}{\bf{y}}$

This corresponds to reflecting ${\bf{y}}$ about the line through the origin perpendicular to ${\bf{u}}$ as shown in the below figure:

![householder](./imgs/householder.png)

Making use of the standard basis axes as a reference point we can choose

$$ u_{1} = \dfrac{{\bf{y}} \pm \|{\bf{y}}\|e_{1}}{\|{\bf{y}} \pm \|{\bf{y}}\|e_{1}}e_{1}$$

This produces a reflection along the $e_{1}$ axis. Then we must construct the Householder matrix:

$$H_{u_{1}} = I - 2u_{1}u_{1}^{T}$$

This matrix is applied in a rotation of $A$:

$$ X_{1} = H_{u_{1}}A $$

The basic idea is that we use the householder reflection to project ${\bf{y}}$ onto the axis orthogonal to ${\bf{u}}$. This projection is propagated throughout the matrix (of column vectors), effectively creating a change of coordinates (a sort of translation). This sets elements below the first diagonal to $0$, and provides a coordinate change to the remaining $n-1$ vectors. Then we take the next vector in the matrix and calculate its reflection against the previously corrected vector, setting the below elements to $0$, propagating to the $n-2$ vectors and so on.

The householder algorithm proceeds for a $m \times n$ matrix as follows:

set $Q = I_{n}$

$for\ i\ in\ num\ columns:$

$\ \ \ \ \ u_{i} = \dfrac{{\bf{y}} \pm \|{\bf{y}}\|e_{i}}{\|{\bf{y}} \pm \|{\bf{y}}\|e_{i}}e_{i}$

$\ \ \ \ \ H_{i} =  I - 2u_{i}u_{i}^{T}$

$\ \ \ \ \ Q = QH_{i}$

Finally you end up with a series of $Q = Q_{n-1}Q_{n-2}...Q_{1}$, giving a final estimate for the real $Q$

### Example:

Let's get the $QR$ decomposition of the two-vector matrix 

$X = \begin{bmatrix}1. & 1.26\\1. & 1.82\\1.& 2.22 \end{bmatrix}$

We first choose a $u$ to take the first column of the matrix to the x-axis:

$u_{1} = \begin{bmatrix}1.\\1.\\1.\end{bmatrix}-\sqrt{3}\begin{bmatrix}1.\\0\\0\end{bmatrix}$

Then we need to normalize it:

$u_{1} = \dfrac{u_{1}}{\|u_{1}\|} =\dfrac{1}{1.5925} \begin{bmatrix}-0.7321.\\1.\\1.\end{bmatrix}$

Now create $H_{u_{1}}$:

$H_{u_{1}} = I - 2u_{1}u_{1}^{T} = \begin{bmatrix}1. & 0 & 0\\0 & 1. & 0\\0 & 0 & 1.\end{bmatrix} - 2\begin{bmatrix} 0.21132487 & -0.28867513 & -0.28867513\\-0.28867513 & 0.39433757 & 0.39433757\\ -0.28867513 & 0.39433757 & 0.39433757\end{bmatrix}$

$H_{u_{1}} = \begin{bmatrix} 0.57735027 & 0.57735027 & 0.57735027 \\ 0.57735027 & 0.21132487 & -0.78867513\\ 0.57735027 & -0.78867513 & 0.21132487\end{bmatrix}$

And we'll start off the creation of Q by allowing the first $Q_{i}$ to be $H_{u_{1}}$:

$Q = IH_{u_{1}}$


The current state of the factorization can be taken with the matrix product of $H_{u_{1}}$ and $X$:

$R = H_{u_{1}}X = \begin{bmatrix} 1.7321 & 3.0600 \\ 0 & -0.6388 \\ 0 & -0.2388\end{bmatrix} $

Now we need to tear down the second column. Note that we don't want to lose the work we've done in the first row, so the next $u$ will not be considering the first row:

$u_{2} = \begin{bmatrix}0\\-0.6388\\-0.2388\end{bmatrix}-0.6820\begin{bmatrix}0\\1.\\0\end{bmatrix}$

$u_{2} = \dfrac{u_{2}}{\|u_{2}\|} = \dfrac{1}{1.3422}\begin{bmatrix}0\\-1.3208\\-0.2388\end{bmatrix}$

$H_{u_{2}} = I - 2u_{2}u_{2}^{T} = \begin{bmatrix} 1 & 0 & 0 \\ 0 & -2.48902528 & -0.63081408\\ 0 & -0.63081408 & 0.88594912\end{bmatrix}$

$R = H_{u_{2}}H_{u_{1}}X = \begin{bmatrix} 1.7321 & 3.0600 \\ 0 & 0.6820 \\ 0 & 0\end{bmatrix} $

$Q = H_{u_{1}}H_{u_{2}} = \begin{bmatrix} 0.57735027 & -0.74295879 & 0.33864273 \\
  0.57735027 & 0.07820619 & -0.81274255 \\
  0.57735027 & 0.6647526 & 0.47409982 \end{bmatrix} $

## TASK:

Below is given an example snippet of the above operations as a major hint. Below that is the code stub you can use to fill out, or try a method of your own. 

In [84]:
A=  np.array([[1,2,3],
            [4,5,6],
            [4,3,2]])

A[0,:] ## print the first column

#print(np.eye(2)) ### this is the identity matrix

#print([(count,item) for count,item in enumerate(range(len(A)))])

array([1, 2, 3])

In [201]:
import numpy as np
import numpy.linalg as LA
eps = 1.0E-10
Q = np.eye(3) ## identity matrix
print(Q[1,:].T,'Second row tranposed Q')
X = np.array([[1., 1.26],[1, 1.82],[1., 2.22]]) ## this is the original matrix
R = np.copy(X) ## create a copy of our original matrix in R

# we are making some shortcuts here, so as not to give everything away

u1 = X[:,0]-LA.norm(X[:,0])*np.array([1.,0.,0.]) # take the first column of our matrix X, subtract the norm of the 
# first column times the x position, the other coordinates we set to zero


u1 = u1/LA.norm(u1) ## normalize your new u1 vector

print(u1, ' first u1')

H1 = np.identity(3)-2.*np.outer(u1, u1) ## get the current state of your Householder matrix, H. Take the full identity 
#minus 2 times the outer product (a matrix) of your u vector

print(np.outer(u1,u1), ' outer product of u1 and u1')

print("h1")
print(H1)
print(' ')

Q = np.dot(Q, H1) ## this is your current Q

print(Q, ' Q')
R = np.dot(H1, R)
print (R, ' R')
print(R[1:,1:],'R sliced')
print(R[1,1],'R 1 1 ')

#continuing on, with more hints

x = H1.T.dot(X)[1:,1] # now you ignore the top i rows (because they've already been solved)

print(x, ' x minus rows')

e = np.zeros_like(H1.T.dot(X)[1:,1]) # you want do reflection only on the elements that haven't been solved for yet
print(e,'e')

e[0] = np.copysign(np.linalg.norm(x), -X[1, 1]) # this is a useful step 
                                            # just to make sure that you've got the right signed norm
print(e,'e after copysign')
print(u,'u before e')
u = x+e
print (u, 'u after  x + e')
v = u / np.linalg.norm(u)

H_i = np.identity(3)
print(np.outer(v,v),'second outer product')

H_i[1:, 1:] -= 2.0 * np.outer(v, v)

print(H_i,'new H')

R = np.dot(H_i, R)
print(R,'-------')
Q = np.dot(Q, H_i)

# Here is a clean way to zero out low values
low_values_indices = R < eps  # Where values are low
R[low_values_indices] = 0  # All low values set to 0

print(Q)
print(' ')
print('and the factorization emerges')
print(R)

[ 0.  1.  0.] Second row tranposed Q
[-0.45970084  0.62796303  0.62796303]  first u1
[[ 0.21132487 -0.28867513 -0.28867513]
 [-0.28867513  0.39433757  0.39433757]
 [-0.28867513  0.39433757  0.39433757]]  outer product of u1 and u1
h1
[[ 0.57735027  0.57735027  0.57735027]
 [ 0.57735027  0.21132487 -0.78867513]
 [ 0.57735027 -0.78867513  0.21132487]]
 
[[ 0.57735027  0.57735027  0.57735027]
 [ 0.57735027  0.21132487 -0.78867513]
 [ 0.57735027 -0.78867513  0.21132487]]  Q
[[  1.73205081e+00   3.05995643e+00]
 [ -1.11022302e-16  -6.38786205e-01]
 [ -1.11022302e-16  -2.38786205e-01]]  R
[[-0.6387862]
 [-0.2387862]] R sliced
-0.638786204584 R 1 1 
[-0.6387862 -0.2387862]  x minus rows
[ 0.  0.] e
[-0.68195797  0.        ] e after copysign
[-1.32074417 -0.2387862 ] u before e
[-1.32074417 -0.2387862 ] u after  x + e
[[ 0.9683472   0.17507399]
 [ 0.17507399  0.0316528 ]] second outer product
[[ 1.          0.          0.        ]
 [ 0.         -0.9366944  -0.35014798]
 [ 0.         -0.3501479

In [204]:
from math import copysign

def householder_reflection(A):
    """Perform QR decomposition of matrix A using Householder reflection."""
    (rows, cols) = np.shape(A)
    # * Initialize orthogonal matrix Q and upper triangular matrix R.
    
    Q = np.identity(len(A))
    R = A
    H_list = []
    iden = np.identity(len(A))
    
    for count , col in enumerate(A):
        
        if count ==0: ## This is correct, returns the correct Q and R
            #print(Q[count,:]," Q here first")
            u1 = A[:,count] - LA.norm(A[:,count])*Q[count,:] ## create u1 by taking the first column and subtracting
            #the norm times the identity for the first row. 
        
            u1 = u1/LA.norm(u1) ##normalize u1
            print(u1, ' u1') ## This is correct
            print(np.outer(u1,u1),'outer product u1 u1 ')
            H_1 = Q-2.*np.outer(u1, u1) ## get out first H1 vector
            
            #H_list.append(H)
            #H_list.append(H)
            
            print(H_1,' first H')
            
            Q = np.dot(Q, H_1) ##get Q, this is multiply the identity Q by H1
            print(Q, ' Q for count == 0')
        
            R = np.dot(H_1, R) ## Next, Get R by taking your current H and multiply by the original matrix
            
            
            #low_values_indices = R < eps  # Where values are low
            #R[low_values_indices] = 0  # All low values set to 0
            print(R, "R for count == 0")
        elif count < cols:
            # here count starts at 1
            print(count,'count')
            #print(H_1, ' full H')
            #print(H_1.T, " H transposed")
            #print(H_1.T.dot(A),'H.T.dot(A)')
            #print(H_1.T.dot(A)[count:,count],'H.T.dot(A) count count')
            
            
            print(np.array(R[count:,count:]), 'R count ccount')
            #temp =[]
            temp= R[count:,count:]
            
            print(temp,'temp')
            
            temp = np.insert(temp,0,0)
            
            print(temp,'new R (temp) should be three elements----------')
            
            print(iden[count,:],'Q count')
            print(R[count,count]*iden[count,:], " r times Q ")
            
            u = temp+R[count,count]*iden[count,:]
            print(u, 'u before norm')

            
            u = u/LA.norm(u)
            print(u, ' u') ## this u is correct 
            
            print(u.T,'u transpose')
            print(np.outer(u,u),'outer product of u and u ') ## this looks good
            
            H_2 = iden-2*np.outer(u,u) ## this is off , off by a little bit
            
            print(H_2, ' this is the new H') ## this looks correct
            
            # Here is a clean way to zero out low values

            
            R = H_2.dot(R)
            
            low_values_indices = R < eps  # Where values are low
            R[low_values_indices] = 0  # All low values set to 0
            #R[]
            print(R, ' this is your R')
            Q = np.dot(Q,H_2) ## this works!


    return (Q, R)

In [208]:
import numpy as np
A = np.array([[1., 1.26],[1, 1.82],[1., 2.22]] ,dtype=float)
householder_reflection(A)
R = Q.T.dot(A)
print(Q)
print(Q.dot(R))

assert np.allclose(Q.dot(R),A)

##IT works!

[-0.45970084  0.62796303  0.62796303]  u1
[[ 0.21132487 -0.28867513 -0.28867513]
 [-0.28867513  0.39433757  0.39433757]
 [-0.28867513  0.39433757  0.39433757]] outer product u1 u1 
[[ 0.57735027  0.57735027  0.57735027]
 [ 0.57735027  0.21132487 -0.78867513]
 [ 0.57735027 -0.78867513  0.21132487]]  first H
[[ 0.57735027  0.57735027  0.57735027]
 [ 0.57735027  0.21132487 -0.78867513]
 [ 0.57735027 -0.78867513  0.21132487]]  Q for count == 0
[[  1.73205081e+00   3.05995643e+00]
 [ -1.11022302e-16  -6.38786205e-01]
 [ -1.11022302e-16  -2.38786205e-01]] R for count == 0
1 count
[[-0.6387862]
 [-0.2387862]] R count ccount
[[-0.6387862]
 [-0.2387862]] temp
[ 0.        -0.6387862 -0.2387862] new R (temp) should be three elements----------
[ 0.  1.  0.] Q count
[-0.        -0.6387862 -0.       ]  r times Q 
[ 0.         -1.27757241 -0.2387862 ] u before norm
[ 0.         -0.98297775 -0.18372464]  u
[ 0.         -0.98297775 -0.18372464] u transpose
[[ 0.         -0.         -0.        ]
 [-0.  

[[ 0.57735027  0.57735027  0.57735027]
 [ 0.57735027  0.21132487 -0.78867513]
 [ 0.57735027 -0.78867513  0.21132487]]  Q
 
 
[[  1.73205081e+00   3.05995643e+00]
 [ -1.11022302e-16  -6.38786205e-01]
 [ -1.11022302e-16  -2.38786205e-01]]  R

In [ ]:
# Final Q nd R - second round

[[ 0.57735027 -0.74295879  0.33864273]
 [ 0.57735027  0.07820619 -0.81274255]
 [ 0.57735027  0.6647526   0.47409982]]
 
and the factorization emerges
[[ 1.73205081  3.05995643]
 [ 0.          0.68195797]
 [ 0.          0.        ]]